# Estudio de los datos de Ventas de una Ferretería

Autor: Diana Chacón Ocariz

## Contexto:

Se trata de una pequeña ferretería que maneja un poco más de 3.000 productos distintos. Poseen un software de gestión genérico que les provee una gran cantidad de reportes, básicamente tablas con números, díficiles de analizar (un reporte puede constar de varias decenas de páginas).


## Objetivos del negocio:

**Tener más visibilidad sobre las ventas para poder mejorar el proceso de compras y la toma de decisiones en general:** 

    - Poder analizar objetivamente las ventas
    - Determinar los productos que podrían entrar en rotura de stock al final de un período
    - Identificar los productos menos vendidos
    - Identificar patrones en el comportamiento de las ventas para poder hacer predicciones de ventas
    

## Objetivos académicos:

    - Estudiar un caso real, con datos reales y cuyo resultado pueda ayudar a alguien a resolver un problema 
    - Demostrar que la ciencia de datos también puede ayudar a las PYMES
    - Conocer y practicar el uso de herramientas de ciencia de datos
    
## Fuentes de datos:

Los datos provienen de reportes sacados del software de gestión de la empresa. Se trata de archivos .xls que contienen sólo los datos de reportes sobre ventas por producto (2021 y 2022) y el stock al final del período. También se utiliza el histórico de la tasa de cambio del dólar para hacer el análisis en base a montos en esta moneda y paliar los problemas de inflación.

# Notebook 1: Carga y Limpieza de Datos

Una vez leídos los datos y luego de una primera limpieza de los DF, los guardaremos en archivos **.parquet** que serán utilizados más tarde en el **EDA**.

También los guardaremos en archivos **.xlsx** que se utilizarán en la creación de reportes en Google Data Studio para failicitar el análisis por parte de los gerentes de la ferretería.

In [1]:
# Librerías utilizadas

import os
import glob
from pathlib import Path

import pandas as pd

In [2]:
BASE_DIR = Path.cwd()
BASE_DIR

PosixPath('/home/diana/Documentos/Ciencia de Datos/Proyecto Ventas')

In [3]:
# Lectura de los archivos y creación de un DF con todos los datos
    
def read_files(FILES, col, long):

    df = pd.DataFrame()

    for filename in glob.glob(f"{BASE_DIR / FILES}"):
        df_aux = pd.read_excel(filename)
        
        info_file = Path(filename).name[:long]
        df_aux[col] = info_file

        df = pd.concat([df, df_aux])

    return df

## Carga de Datos de las Ventas

In [4]:
FILES_VENTAS = 'datos/in/*art*.xls'
FILES_VENTAS

'datos/in/*art*.xls'

In [5]:
[Path(filename).name for filename in glob.glob(f"{BASE_DIR / FILES_VENTAS}")]

['fa_art_2021.xls', 'ne_art_2021.xls', 'fa_art_2022.xls', 'ne_art_2022.xls']

In [6]:
%%time

df = read_files(FILES_VENTAS, 'tipo', 2)

df

CPU times: user 2.36 s, sys: 63.1 ms, total: 2.42 s
Wall time: 2.54 s


,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,tipo
0,00001,NaN,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
1,0000000702,1,2021-01-21 10:02:00,18637949,14.0,1.00,2.0,UNI,5.40,5.40,0.0,0.0,5.40,fa
2,0000000703,1,2021-01-21 10:24:00,19339285,7.0,1.00,1.0,UNI,2.70,2.70,0.0,0.0,2.70,fa
3,0000001508,1,2021-02-09 11:47:00,9122407,7.0,1.00,1.0,UNI,3.24,3.24,0.0,0.0,3.24,fa
4,0000001769,1,2021-02-17 12:09:00,10740296,10.0,1.00,1.0,UNI,3.24,3.24,0.0,0.0,3.24,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3517,GEN1,NaN,GENERICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne
3518,0000006182,1,2022-01-05 16:35:00,27232277.0,13.0,1.00,1.0,UNI,97.50,97.50,0.0,0.0,97.50,ne
3519,0000006546,1,2022-01-14 08:47:00,18018023.0,7.0,1.00,2.0,UNI,4.90,9.80,0.0,0.0,9.80,ne
3520,Sub-Totales:,3,107.3,0.0,0.0,107.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne


In [7]:
df.describe()

,Vendedor,Almacén,Cantidad,Precio Unitario,Monto Base,I.V.A.,Otros,Neto
count,33640.000000,33640.000000,28925.000000,28925.000000,28925.000000,28925.000000,28925.000000,28925.000000
mean,8.006907,81.789050,4.566491,19.517480,26.960769,0.692567,0.000006,26.880727
std,4.591927,4574.323653,21.901486,97.215986,137.228900,7.324589,0.000764,113.978329
min,0.000000,0.010000,0.020000,0.010000,0.010000,0.000000,0.000000,0.010000
25%,7.000000,1.000000,1.000000,1.980000,2.600000,0.000000,0.000000,2.690000
50%,10.000000,1.000000,1.000000,5.555550,7.500000,0.000000,0.000000,7.650000
75%,11.000000,1.000000,3.000000,14.588750,20.500000,0.000000,0.000000,21.000000
max,14.000000,618970.850000,2000.000000,5287.500000,11340.010000,846.000000,0.120000,6133.500000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38351 entries, 0 to 3521
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Número            38351 non-null  object 
 1   Reng              34207 non-null  object 
 2   Emisión           38351 non-null  object 
 3   Cliente           33640 non-null  object 
 4   Vendedor          33640 non-null  float64
 5   Almacén           33640 non-null  float64
 6   Cantidad          28925 non-null  float64
 7   Unid.             28925 non-null  object 
 8   Precio Unitario   28925 non-null  float64
 9   Monto Base        28925 non-null  float64
 10  I.V.A.            28925 non-null  float64
 11  Otros             28925 non-null  float64
 12  Neto              28925 non-null  float64
 13  tipo              38351 non-null  object 
dtypes: float64(8), object(6)
memory usage: 4.4+ MB


## Limpieza y transformación de los datos:

Aunque los datos no tienen ningún formato especial de Excel, tienen la forma de un reporte con totales, subtotales y datos agrupados por producto

- **Eliminación de totales:** Eliminamos las lineas que continen "total" ya que son los totales y subtotales de los reportes
- **Eliminación de información no relevante:** Conservaremos solo las siguientes columnas: Número, Emisión, Cliente, Vendedor, Cantidad, Neto y Tipo 
- **Construcción del DF definitvo:** Recorreremos el DF para recuperar la información por producto y crearemos un nuevo DF con los datos definitivos
- **Cambio tipos columnas:** Cambiamos el tipo a la columna Vendedor para que sea de tipo entero. Transformamos el campo de fecha para que sea de tipo datetime. Transformamos las columnas num y cliente a str

### Eliminación de totales y subtotales

In [9]:
# Buscamos las filas de totales y subtotales para eliminarlas
df[df.Número.str.contains('Totales')] 

,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,tipo
19,Sub-Totales:,19,81.74,7.85,0.0,89.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
56,Sub-Totales:,154,376.3,40.5,0.0,416.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
60,Sub-Totales:,2,25.6,4.1,0.0,29.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
63,Sub-Totales:,1,12.27,0,0.0,12.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
103,Sub-Totales:,59.25,716.74,88.87,0.0,805.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509,Sub-Totales:,4,135.2,0.0,0.0,135.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne
3513,Sub-Totales:,2,56.15,0.0,0.0,56.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne
3516,Sub-Totales:,1,6.5,0.0,0.0,6.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne
3520,Sub-Totales:,3,107.3,0.0,0.0,107.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne


In [10]:
df_aux = df[~df.Número.str.contains('Totales')]
df_aux

,Número,Reng,Emisión,Cliente,Vendedor,Almacén,Cantidad,Unid.,Precio Unitario,Monto Base,I.V.A.,Otros,Neto,tipo
0,00001,NaN,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa
1,0000000702,1,2021-01-21 10:02:00,18637949,14.0,1.0,2.0,UNI,5.40,5.40,0.0,0.0,5.40,fa
2,0000000703,1,2021-01-21 10:24:00,19339285,7.0,1.0,1.0,UNI,2.70,2.70,0.0,0.0,2.70,fa
3,0000001508,1,2021-02-09 11:47:00,9122407,7.0,1.0,1.0,UNI,3.24,3.24,0.0,0.0,3.24,fa
4,0000001769,1,2021-02-17 12:09:00,10740296,10.0,1.0,1.0,UNI,3.24,3.24,0.0,0.0,3.24,fa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,06383,NaN,CONFITERIA TRIDENT TOTAL YERBABUENA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne
3515,0000006535,3,2022-01-13 16:05:00,9333194.0,13.0,1.0,1.0,UNI,6.50,6.50,0.0,0.0,6.50,ne
3517,GEN1,NaN,GENERICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ne
3518,0000006182,1,2022-01-05 16:35:00,27232277.0,13.0,1.0,1.0,UNI,97.50,97.50,0.0,0.0,97.50,ne


### Eliminación de columnas no relevantes

In [11]:
# Cambiamos el nombre de las columnas
df_aux.columns

Index(['Número', 'Reng', 'Emisión ', 'Cliente ', 'Vendedor', 'Almacén',
       'Cantidad', 'Unid.', 'Precio Unitario ', 'Monto Base', 'I.V.A.',
       'Otros', 'Neto', 'tipo'],
      dtype='object')

In [12]:
cols = ['num', 'reng', 'fecha_comp', 'cliente', 'vendedor', 'almacen', 'cantidad',
       'und', 'precio', 'base', 'iva', 'otros', 'neto', 'tipo']
df_aux.columns = cols
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33636 entries, 0 to 3519
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   num         33636 non-null  object 
 1   reng        29492 non-null  object 
 2   fecha_comp  33636 non-null  object 
 3   cliente     28925 non-null  object 
 4   vendedor    28925 non-null  float64
 5   almacen     28925 non-null  float64
 6   cantidad    28925 non-null  float64
 7   und         28925 non-null  object 
 8   precio      28925 non-null  float64
 9   base        28925 non-null  float64
 10  iva         28925 non-null  float64
 11  otros       28925 non-null  float64
 12  neto        28925 non-null  float64
 13  tipo        33636 non-null  object 
dtypes: float64(8), object(6)
memory usage: 3.8+ MB


In [13]:
df_aux = df_aux.loc[:,['num', 'fecha_comp', 'cliente', 'vendedor', 'cantidad', 'neto', 'tipo']]
df_aux


,num,fecha_comp,cliente,vendedor,cantidad,neto,tipo
0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,NaN,NaN,NaN,NaN,fa
1,0000000702,2021-01-21 10:02:00,18637949,14.0,2.0,5.40,fa
2,0000000703,2021-01-21 10:24:00,19339285,7.0,1.0,2.70,fa
3,0000001508,2021-02-09 11:47:00,9122407,7.0,1.0,3.24,fa
4,0000001769,2021-02-17 12:09:00,10740296,10.0,1.0,3.24,fa
...,...,...,...,...,...,...,...
3514,06383,CONFITERIA TRIDENT TOTAL YERBABUENA,NaN,NaN,NaN,NaN,ne
3515,0000006535,2022-01-13 16:05:00,9333194.0,13.0,1.0,6.50,ne
3517,GEN1,GENERICO,NaN,NaN,NaN,NaN,ne
3518,0000006182,2022-01-05 16:35:00,27232277.0,13.0,1.0,97.50,ne


### Construcción del DF definitivo

Los datos en el DF están agrupados por producto: Una línea tiene la información sobre el producto y las siguientes son las facturas que incluyen el producto. 

El objetivo es obtener un DF con la siguiente información por cada línea:

    - num: Número de factura (columna num)
    - fecha: Fecha de la factura (columna fecha)
    - cliente: Código del cliente (columna cliente)
    - vendedor: Código del vendedor (columna vendedor)
    - cod: Código del producto (columna num cuando el largo <=5 )
    - producto: Descripción del producto (columna fecha)
    - cantidad: Cantidad de producto en la factura (columna cantidad)
    - monto: Monto neto del producto en la factura (columna neto)
    - tipo: Tipo de factura (columna tipo)
    
Para eso, recorremos el DF y lo vamos construyendo.

In [14]:
%%time

result = []

cod = ''
prod = ''

for index, row in df_aux.iterrows(): 
    if len(row['num']) <= 5 :
        cod = row['num']
        prod = row['fecha_comp']
    else:
        dic = {}
        dic['num'] = row['num']
        dic['fecha_comp'] = row['fecha_comp']
        dic['cliente'] = row['cliente']
        dic['vendedor'] = row['vendedor']
        dic['cod'] = cod
        dic['producto'] = prod
        dic['cantidad'] = row['cantidad']
        dic['monto'] = row['neto']   
        dic['tipo'] = row['tipo'] 
        
        result.append(dic)


CPU times: user 3.51 s, sys: 29.7 ms, total: 3.54 s
Wall time: 3.87 s


In [15]:
df_ventas = pd.DataFrame(result)
df_ventas.sample(30)

,num,fecha_comp,cliente,vendedor,cod,producto,cantidad,monto,tipo
13540,0000004185,2021-05-05 10:50:00,14281169,3.0,06278,COCINA ELECTRICA 2 HORNILLAS HACEB COLOMBIANA,1.0,123.08,fa
24225,0000002845,2021-09-21 11:34:00,15862718.0,7.0,04442,"HOJA PARA SEGUETA 18TPI 12"" SPEED",12.0,14.76,ne
12880,0000005477,2021-06-23 10:55:00,2812834,3.0,05142,BOMBILLO 6W LED PANEL CUADRADO SOBREPONER VERT...,2.0,28.38,fa
12680,0000003053,2021-03-26 11:12:00,16788644,11.0,05102,COPIA DE LLAVE,1.0,1.60,fa
10264,0000002914,2021-03-23 09:23:00,9333194,7.0,03813,CINTA PERIMETRAL PRECAUCION AMARILLA TRUPER PO...,12.0,2.04,fa
21836,0000000818,2021-07-17 08:59:00,20717627.0,11.0,02588,"LLAVE PLASTICA 3/4"" ROSCADA",4.0,17.76,ne
6626,0000001937,2021-02-22 13:22:00,5346238,1.0,01257,"NIPLE PLASTICO 2"" * 15 CTMS URAPLAST",4.0,14.00,fa
19441,0000004368,2021-11-11 11:16:00,12889648.0,7.0,01130,"ANILLO 2"" PLASTICO SCHEDULE 80 ROSCADO AZUL UR...",1.0,6.90,ne
13457,0000002324,2021-03-04 15:55:00,310060940,10.0,06162,"BISAGRA ARMILLAR 3"" (POR UNIDAD)",4.0,17.63,fa
4138,0000000411,2021-01-14 16:08:00,24152112,11.0,00776,BOLSA PLASTICA PEQUEÑA 10KRGS (USO INTERNO) UN...,1.0,0.02,fa


In [16]:
df_ventas.describe()

,vendedor,cantidad,monto
count,28925.000000,28925.000000,28925.000000
mean,9.312083,4.566491,26.880727
std,3.516934,21.901486,113.978329
min,1.000000,0.020000,0.010000
25%,7.000000,1.000000,2.690000
50%,10.000000,1.000000,7.650000
75%,11.000000,3.000000,21.000000
max,14.000000,2000.000000,6133.500000


In [17]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28925 entries, 0 to 28924
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   num         28925 non-null  object        
 1   fecha_comp  28925 non-null  datetime64[ns]
 2   cliente     28925 non-null  object        
 3   vendedor    28925 non-null  float64       
 4   cod         28925 non-null  object        
 5   producto    28925 non-null  object        
 6   cantidad    28925 non-null  float64       
 7   monto       28925 non-null  float64       
 8   tipo        28925 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 2.0+ MB


In [18]:
# Transformamos los tipos de columnas para que se guarden con
# el tipo correcto

df_ventas.num = df_ventas.num.astype(str)
df_ventas.cod = df_ventas.cod.astype(str)
df_ventas.vendedor = df_ventas.vendedor.astype(int)
df_ventas.cliente = df_ventas.cliente.astype(str)

df_ventas.fecha_comp = pd.to_datetime(df_ventas.fecha_comp)
df_ventas['fecha'] = pd.to_datetime(df_ventas.fecha_comp.dt.date)


df_ventas


,num,fecha_comp,cliente,vendedor,cod,producto,cantidad,monto,tipo,fecha
0,0000000702,2021-01-21 10:02:00,18637949,14,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,2.0,5.40,fa,2021-01-21
1,0000000703,2021-01-21 10:24:00,19339285,7,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,2.70,fa,2021-01-21
2,0000001508,2021-02-09 11:47:00,9122407,7,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,3.24,fa,2021-02-09
3,0000001769,2021-02-17 12:09:00,10740296,10,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,3.24,fa,2021-02-17
4,0000001922,2021-02-22 11:30:00,24777484,7,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,3.42,fa,2021-02-22
...,...,...,...,...,...,...,...,...,...,...
28920,0000006357,2022-01-10 10:43:00,9331102.0,13,06381,MAXICAL 10 KILOS,1.0,28.22,ne,2022-01-10
28921,0000006460,2022-01-12 11:02:00,17527596.0,7,06381,MAXICAL 10 KILOS,1.0,27.93,ne,2022-01-12
28922,0000006535,2022-01-13 16:05:00,9333194.0,13,06383,CONFITERIA TRIDENT TOTAL YERBABUENA,1.0,6.50,ne,2022-01-13
28923,0000006182,2022-01-05 16:35:00,27232277.0,13,GEN1,GENERICO,1.0,97.50,ne,2022-01-05


## Tasa de cambio

Para mitigar los efectos de la inflación, los montos, originalmente en Bolívares (Bs), serán transformados a montos en $USD. Para eso, utilizamos un archivo CSV con todas las tasas de cambio diarias. Luego asignamos la tasa correspondiente a cada registro según la fecha.

El archivo CSV con las tasas de cambio lo creamos a partir de un [scraper](https://github.com/dchaconoca/proyecto-ventas/blob/master/scraper_tasa_dolar.ipynb)

In [19]:
FILE_DOLAR = 'datos/in/tasa_dolar.csv'

df_dolar = pd.read_csv(f"{BASE_DIR / FILE_DOLAR}", sep=';')
df_dolar

,fecha,alta,baja
0,02-02-2022,4.69,4.70
1,01-02-2022,4.69,4.74
2,31-01-2022,4.73,4.75
3,30-01-2022,4.73,4.74
4,29-01-2022,4.73,4.75
...,...,...,...
806,19-11-2019,25931.00,33034.00
807,18-11-2019,29027.00,30849.00
808,17-11-2019,28277.00,29509.00
809,16-11-2019,28430.00,29229.00


In [20]:
# Convertimos la columna de fecha
# y extraemos solo los datos a partir del 2021

df_dolar['fecha'] = pd.to_datetime(df_dolar['fecha'], dayfirst=True)
df_dolar = df_dolar.query(' fecha > "2020/12/31" ')
df_dolar

,fecha,alta,baja
0,2022-02-02,4.69,4.70
1,2022-02-01,4.69,4.74
2,2022-01-31,4.73,4.75
3,2022-01-30,4.73,4.74
4,2022-01-29,4.73,4.75
...,...,...,...
393,2021-01-05,1178316.62,1303357.48
394,2021-01-04,1051007.98,1201219.02
395,2021-01-03,1036015.05,1088223.91
396,2021-01-02,1042518.04,1099758.03


In [21]:
# Aplicamos la reconversión (eliminar 6 ceros a partir del 01-10 o para montos mayores de 1000)
# Para el momento de la reconversión, el precio del $ sobrepasaba el 1.000.000 de Bs
def conversion(x):
    return (x/1000000 if x>1000 else x)

df_dolar['tasa_dolar'] = df_dolar.apply(lambda row: conversion(row['baja']), axis = 1)
df_dolar

/home/diana/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,fecha,alta,baja,tasa_dolar
0,2022-02-02,4.69,4.70,4.700000
1,2022-02-01,4.69,4.74,4.740000
2,2022-01-31,4.73,4.75,4.750000
3,2022-01-30,4.73,4.74,4.740000
4,2022-01-29,4.73,4.75,4.750000
...,...,...,...,...
393,2021-01-05,1178316.62,1303357.48,1.303357
394,2021-01-04,1051007.98,1201219.02,1.201219
395,2021-01-03,1036015.05,1088223.91,1.088224
396,2021-01-02,1042518.04,1099758.03,1.099758


In [22]:
df_dolar = df_dolar.loc[:, ['fecha', 'tasa_dolar']]
df_dolar

,fecha,tasa_dolar
0,2022-02-02,4.700000
1,2022-02-01,4.740000
2,2022-01-31,4.750000
3,2022-01-30,4.740000
4,2022-01-29,4.750000
...,...,...
393,2021-01-05,1.303357
394,2021-01-04,1.201219
395,2021-01-03,1.088224
396,2021-01-02,1.099758


In [23]:
df_ventas_fin = pd.merge(df_ventas, df_dolar, on='fecha')

df_ventas_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28925 entries, 0 to 28924
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   num         28925 non-null  object        
 1   fecha_comp  28925 non-null  datetime64[ns]
 2   cliente     28925 non-null  object        
 3   vendedor    28925 non-null  int64         
 4   cod         28925 non-null  object        
 5   producto    28925 non-null  object        
 6   cantidad    28925 non-null  float64       
 7   monto       28925 non-null  float64       
 8   tipo        28925 non-null  object        
 9   fecha       28925 non-null  datetime64[ns]
 10  tasa_dolar  28925 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 2.6+ MB


In [24]:
# Calculamos el monto (total facturado) en dólares a la tasa del día
def calculo_precio_dolar(monto, tasa):
    return (monto/tasa)

df_ventas_fin['monto_dolar'] = df_ventas_fin.apply(lambda row: calculo_precio_dolar(row['monto'], row['tasa_dolar']), 
                                              axis = 1)

df_ventas_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28925 entries, 0 to 28924
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   num          28925 non-null  object        
 1   fecha_comp   28925 non-null  datetime64[ns]
 2   cliente      28925 non-null  object        
 3   vendedor     28925 non-null  int64         
 4   cod          28925 non-null  object        
 5   producto     28925 non-null  object        
 6   cantidad     28925 non-null  float64       
 7   monto        28925 non-null  float64       
 8   tipo         28925 non-null  object        
 9   fecha        28925 non-null  datetime64[ns]
 10  tasa_dolar   28925 non-null  float64       
 11  monto_dolar  28925 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(1), object(5)
memory usage: 2.9+ MB


## Almacenamiento del DF

In [25]:
%%time
# Guardamos el DF limpio para su análisis posterior

df_ventas_fin.to_parquet(f"{BASE_DIR / 'datos/out/ventas.parquet'}", 
                    compression='GZIP',
                    engine='pyarrow')

df_ventas_fin.to_csv(f"{BASE_DIR / 'datos/out/ventas.csv'}", sep=';')


# Guardamos el DF en un archivo Excel para utilizar los datos en Google Data Studio
df_ventas_fin.to_excel(f"{BASE_DIR / 'datos/out/ventas.xlsx'}", index = False)

CPU times: user 10.6 s, sys: 191 ms, total: 10.8 s
Wall time: 11.4 s


## Carga de Datos del Inventario

Datos sobre el inventario al final del período. Además incluye la clasificación de los productos en categorías.

Con esta información podemos determinar cuáles productos podrían entrar en rotura de stock en función de lo que se ha vendido y lo que queda en stock

In [26]:
FILE_STOCK = 'datos/in/*inv.xls'

df_stock = read_files(FILE_STOCK, 'fecha', 10)
df_stock

,Código,Descripción,Descrip linea/ validada,SActual,stock real 1,Ajuste(E/S),fecha
0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,GENERALES,16.5,NaN,NaN,2022-02-01
1,00002,MANTO ASFALTICO 10MTSx3.2MM BITUPLAST,NO ASIGNADO,3.0,NaN,NaN,2022-02-01
2,00003,"TACO 1"" BASE REDONDA PVC DURA PARA PATAS DE MU...",GENERALES,13.0,NaN,NaN,2022-02-01
3,00004,MUEBLE ACCESORIOS DE BAÑO HIERRO FORJADO(UNIDAD),HIERRO FORJADO JC Y ABRAZADERAS METALICAS RIEGO,2.0,NaN,NaN,2022-02-01
4,00005,"NIPLE PLASTICO 1"" * 13CMS","NIPLES DE HIERRO GALVANIZADO, CROMADO Y PLASTICOS",34.0,NaN,NaN,2022-02-01
...,...,...,...,...,...,...,...
3547,06382,CONFITERIA CARAMELO CHAO X2,CHUCHERIAS,280.0,NaN,NaN,2022-02-01
3548,06383,CONFITERIA TRIDENT TOTAL YERBABUENA,CHUCHERIAS,4.0,NaN,NaN,2022-02-01
3549,06384,HIDROJET DE ALTA PRESION 1400W INGCO,GENERALES,1.0,NaN,NaN,2022-02-01
3550,06385,CONFITERIA SUSY MAXI 50GR,CHUCHERIAS,1.0,NaN,NaN,2022-02-01


### Limpieza del archivo de inventario

In [27]:
cols = ['cod', 'producto', 'categoria', 'stock', 'stock2', 'ajuste', 'fecha']
df_stock.columns = cols
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3552 entries, 0 to 3551
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   cod        3552 non-null   object 
 1   producto   3552 non-null   object 
 2   categoria  3551 non-null   object 
 3   stock      3551 non-null   float64
 4   stock2     0 non-null      float64
 5   ajuste     0 non-null      float64
 6   fecha      3552 non-null   object 
dtypes: float64(3), object(4)
memory usage: 194.4+ KB


In [28]:
# Guardamos solo la información que nos interesa
df_stock = df_stock.loc[:,['cod', 'producto', 'categoria', 'stock', 'fecha']]
df_stock

,cod,producto,categoria,stock,fecha
0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,GENERALES,16.5,2022-02-01
1,00002,MANTO ASFALTICO 10MTSx3.2MM BITUPLAST,NO ASIGNADO,3.0,2022-02-01
2,00003,"TACO 1"" BASE REDONDA PVC DURA PARA PATAS DE MU...",GENERALES,13.0,2022-02-01
3,00004,MUEBLE ACCESORIOS DE BAÑO HIERRO FORJADO(UNIDAD),HIERRO FORJADO JC Y ABRAZADERAS METALICAS RIEGO,2.0,2022-02-01
4,00005,"NIPLE PLASTICO 1"" * 13CMS","NIPLES DE HIERRO GALVANIZADO, CROMADO Y PLASTICOS",34.0,2022-02-01
...,...,...,...,...,...
3547,06382,CONFITERIA CARAMELO CHAO X2,CHUCHERIAS,280.0,2022-02-01
3548,06383,CONFITERIA TRIDENT TOTAL YERBABUENA,CHUCHERIAS,4.0,2022-02-01
3549,06384,HIDROJET DE ALTA PRESION 1400W INGCO,GENERALES,1.0,2022-02-01
3550,06385,CONFITERIA SUSY MAXI 50GR,CHUCHERIAS,1.0,2022-02-01


In [29]:
# Eliminamos las filas con NaN (total)
# Transformamos los tipos de columnas para que se guarden con
# el tipo correcto
df_stock.dropna(inplace=True)
df_stock.cod = df_stock.cod.astype(str)
df_stock.fecha = pd.to_datetime(df_stock.fecha)

df_stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3551 entries, 0 to 3550
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   cod        3551 non-null   object        
 1   producto   3551 non-null   object        
 2   categoria  3551 non-null   object        
 3   stock      3551 non-null   float64       
 4   fecha      3551 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 166.5+ KB


In [30]:
%%time
# Guardamos el DF limpio para su análisis posterior

df_stock.to_parquet(f"{BASE_DIR / 'datos/out/stock.parquet'}", 
                    compression='GZIP',
                    engine='pyarrow')

df_stock.to_csv(f"{BASE_DIR / 'datos/out/stock.csv'}", sep=';')


# Guardamos el DF en un archivo Excel para utilizar los datos en Google Data Studio
df_stock.to_excel(f"{BASE_DIR / 'datos/out/stock.xlsx'}", index = False)


CPU times: user 842 ms, sys: 4.17 ms, total: 846 ms
Wall time: 884 ms


In [31]:
df_ventas_stock = pd.merge(df_ventas_fin, df_stock, on='cod', how='outer')

df_ventas_stock = df_ventas_stock.loc[:,['num', 'fecha_comp', 'vendedor', 'cod', 'producto_x', 'cantidad', 'monto_dolar', 'categoria', 'stock', 'fecha_y']]

df_ventas_stock.columns = ['num', 'fecha', 'vendedor', 'cod', 'producto', 'cantidad', 'monto_dolar', 'categoria', 'stock', 'fecha_stock']

In [32]:
df_ventas_stock

,num,fecha,vendedor,cod,producto,cantidad,monto_dolar,categoria,stock,fecha_stock
0,0000000702,2021-01-21 10:02:00,14.0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,2.0,3.021055,GENERALES,16.5,2022-02-01
1,0000000703,2021-01-21 10:24:00,7.0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,1.510527,GENERALES,16.5,2022-02-01
2,0000001508,2021-02-09 11:47:00,7.0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,1.814189,GENERALES,16.5,2022-02-01
3,0000001769,2021-02-17 12:09:00,10.0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,1.857821,GENERALES,16.5,2022-02-01
4,0000001922,2021-02-22 11:30:00,7.0,00001,PEGA DE CONTACTO/PEGA ZAPATERA (90 ML) ENVASADO,1.0,1.822781,GENERALES,16.5,2022-02-01
...,...,...,...,...,...,...,...,...,...,...
30129,NaN,NaT,NaN,06358,NaN,NaN,NaN,"NIPLES DE HIERRO GALVANIZADO, CROMADO Y PLASTICOS",0.0,2022-02-01
30130,NaN,NaT,NaN,06367,NaN,NaN,NaN,LLAVES O VALVULAS METALICAS,3.0,2022-02-01
30131,NaN,NaT,NaN,06371,NaN,NaN,NaN,GENERALES,6.0,2022-02-01
30132,NaN,NaT,NaN,06384,NaN,NaN,NaN,GENERALES,1.0,2022-02-01


In [33]:
# Guardamos el DF en un archivo Excel para utilizar los datos en Google Data Studio
df_ventas_stock.to_excel(f"{BASE_DIR / 'datos/out/ventas_stock.xlsx'}", index = False)